<a href="https://colab.research.google.com/github/roshanappa/Deep-Learning-Models./blob/master/Second_Artificial_Neural_Network_Credit_Card_Fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.feature_selection import VarianceThreshold

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [0]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [0]:
data = pd.read_csv('/content/creditcard.csv')

In [0]:
pd.pandas.set_option('display.max_columns',None)

In [0]:
data.head()

In [0]:

data.tail()

In [0]:
data.columns

In [0]:
data.info()

In [0]:
data.describe()

## Data PreProcessing.

In [0]:
x =data.drop(columns=['Class'])
y = data['Class']

In [0]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.80,stratify=y,random_state=42)

In [0]:
x_train.shape,y_train.shape

In [0]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test  = scaler.transform(x_test)

In [0]:
y_train.value_counts()

In [0]:
y_train.value_counts().plot(kind='bar')
plt.show()

## Model Creation.

In [0]:
model = Sequential()
model.add(Dense(500,activation='relu',kernel_initializer='uniform',input_dim=X_train.shape[1]))
model.add(Dropout(0.40))
model.add(Dense(200,activation='relu',kernel_initializer='uniform'))
model.add(Dropout(0.40))
model.add(Dense(60,activation='relu',kernel_initializer='uniform'))
model.add(Dropout(0.40))

model.add(Dense(1,activation='sigmoid',kernel_initializer='uniform'))

In [0]:
model.summary()

In [0]:
from tensorflow.keras.optimizers import SGD

In [0]:
model.compile(optimizer=SGD(learning_rate=0.01,momentum=0.9),loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
history = model.fit(X_train,y_train,batch_size=20,epochs=10,verbose=1,validation_split=(0.20))

In [0]:
val_loss,val_accuracy=model.evaluate(X_test,y_test,batch_size=20,verbose=1)

In [0]:
y_prediction = model.predict_classes(X_test)

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
mat = confusion_matrix(y_test,y_prediction)
mat

In [0]:
sns.heatmap(mat,annot=True,fmt='d',linecolor='black',linewidths=0.4,cbar=False,xticklabels=['NOt Froud','Froud'],yticklabels=['NOt Froud','Froud'],cmap=plt.get_cmap('Set1'))
plt.show()

In [0]:
from sklearn.metrics import accuracy_score,classification_report,auc,roc_curve

In [0]:
print('Accuarcy Score :-',accuracy_score(y_test,y_prediction))

In [0]:
print(classification_report(y_test,y_prediction))

## Cross Validate The Model.

In [0]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
def set_model():
  model = Sequential()
  model.add(Dense(500,activation='relu',kernel_initializer='uniform',input_dim=X_train.shape[1]))
  model.add(Dropout(0.40))
  model.add(Dense(200,activation='relu',kernel_initializer='uniform'))
  model.add(Dropout(0.40))
  model.add(Dense(60,activation='relu',kernel_initializer='uniform'))
  model.add(Dropout(0.40))
  model.add(Dense(1,activation='sigmoid',kernel_initializer='uniform'))
  model.compile(optimizer=SGD(learning_rate=0.01,momentum=0.9),loss='binary_crossentropy',metrics=['accuracy'])
  
  return model
model 

classifier = KerasClassifier(set_model,batch_size=20,epochs=10)
accuracy = cross_val_score(classifier,X_train,y_train,cv=10,n_jobs=-1)

In [0]:
accuracy

In [0]:
print('mean of the accuracies :-',accuracy.mean())
print('variance of the accuracies :-',accuracy.std())

- From the above we can say that variance between the accuracies is very low as compare to the other.
- but the mean of cross validation accuaracies is pretty much closer to the actual predited accuaracies.

## Learning Curve

In [0]:
history.history

In [0]:
plt.figure(figsize=(6,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Model Accuracy','Validation Accuracy'])
plt.xlabel('Epoches')
plt.ylabel('Accuarcies')
plt.title('Model Accuracies vs Validation Accuracies')
plt.show()

plt.figure(figsize=(6,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Model loss','Validation loss'])
plt.xlabel('Epoches')
plt.ylabel('losses')
plt.title('Model losses vs Validation losses')
plt.show()

## AUC and ROC Curve.

In [0]:
probablity = model.predict_proba(X_test)

In [0]:
fpr,tpr,thresh = roc_curve(y_test,probablity)

In [0]:
auc = round(auc(fpr,tpr)*100,1)

In [0]:
auc

In [0]:
roc_data = pd.DataFrame(dict(fpr=fpr,tpr=tpr))

In [0]:
from plotnine import ggplot,ggtitle,geom_abline,geom_area,geom_line,aes

In [0]:
ggplot(roc_data,aes(x='fpr',y='tpr'))+geom_line()+geom_abline(alpha=0.9,color='green',linetype='dashed')+geom_area(alpha=0.3)+ggtitle('AUC-ROC Curve with value AUC = {}'.format(auc))

## Thank You !!